In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# Importing Our Data

In [47]:
## I will first be importing my pivoted catcherr csv in order to add the
## features needed to save the final csv
df = pd.read_csv('https://raw.githubusercontent.com/oneilltp18/GABBERT/master/wide_receivers/pivot_catcherr.csv')

#need to drop the unnamed first column when we import csv's
df.drop('Unnamed: 0', axis = 1, inplace = True)

# going to set the index to the name of the player
df.set_index(df.name, inplace = True)

# need to drop the name column as it is now the index
df.drop('name', axis = 1, inplace = True)

## I also need to drop a couple year 4 stats that snuck in for some reason
df.drop(['receptions_3', 'rec_yards_3'], axis = 1, inplace = True)

# Also I will create categories for player season_3 ratings (bins)
bins = [-1, 10, 30, 65, 200]
labels = ['below average', 'league_average', 'quality starter', 'all_pro']
df['categories'] =  pd.cut(df['compilation_3'], bins, labels=labels)


###### I am now rewriting over the pivot_catcherr csv with our final version
df.to_csv('/Users/TerryONeill/Terry_git/Capstone/GABBERT/wide_receivers/pivot_catcherr.csv')

df.head()

,season_0,season_1,season_2,age_0,age_1,age_2,weight_0,weight_1,weight_2,bmi_0,...,compilation_0,compilation_1,compilation_2,compilation_3,avg_starts,dpis,dpi_yards,year_1_growth,year_2_growth,categories
name,,,,,,,,,,,,,,,,,,,,,
A.J. Green,2011.0,2012.0,2013.0,23.0,24.0,25.0,205.0,205.0,205.0,25.0,...,53.006410,98.471570,82.935491,59.741763,1.000000,0.0,0.0,9.264454,-13.711468,quality starter
A.J. Jenkins,2012.0,2013.0,2014.0,23.0,24.0,25.0,192.0,192.0,192.0,26.0,...,0.000000,0.597268,0.000000,0.000000,0.020833,0.0,0.0,0.121706,-0.527123,below average
Aaron Brown,2009.0,2010.0,2011.0,24.0,25.0,26.0,196.0,196.0,196.0,26.6,...,1.007213,0.000000,0.000000,0.000000,0.044444,0.0,0.0,-0.205240,0.000000,below average
Aaron Halterman,2007.0,2008.0,2009.0,25.0,26.0,27.0,255.0,255.0,255.0,30.2,...,0.030196,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.006153,0.000000,below average
Aaron Moorehead,2003.0,2004.0,2005.0,23.0,24.0,25.0,200.0,200.0,200.0,25.0,...,0.240777,0.025880,0.222518,0.274562,0.000000,0.0,0.0,-0.043790,0.173544,below average


In [48]:
## I want to create a list of the column names associated with a players
## first and second year stats, the following loop will go through and add
## all these stats to the list year_12

year_12 = []
for col in df:
    if col == 'year_1_growth':
        pass
    else:
        if '_0' in col:
            year_12.append(col)
        if '_1' in col:
            year_12.append(col)
            
            
## I also want to create a list of the rest of the col names (year3 data)
year_3 = []
for col in df:
    if col not in year_12:
        year_3.append(col)
        
year_3

['season_2',
 'age_2',
 'weight_2',
 'bmi_2',
 'games_2',
 'rush_atts_2',
 'rush_yds_2',
 'rush_y/a_2',
 'rush_tds_2',
 'rush_ypg_2',
 'targets_2',
 'receptions_2',
 'rec_yards_2',
 'yards/reception_2',
 'rec_tds_2',
 'rec_ypg_2',
 'ctch_pct_2',
 'y/tgt_2',
 'fumbles_2',
 'fumbles_recovered_2',
 'fum_ret_yds_2',
 'fum_tds_2',
 'forced_fumbles_2',
 'pro_bowls_2',
 'all_pros_2',
 '100yd_gms_2',
 'first_down_ctchs_2',
 'first_down_ctchpct_2',
 'long_ctch_2',
 'drops_2',
 'EYds_2',
 'DVOA_2',
 'DYAR_2',
 '40 Yard_2',
 'rookie_age_2',
 'rookie_season_2',
 'height_inches_2',
 'start_ratio_2',
 'dpis_drawn_2',
 'dpi_yards_2',
 'pct_team_tgts_2',
 'pct_team_receptions_2',
 'pct_of_team_passyards_2',
 'pct_team_touchdowns_2',
 'dropK_2',
 'yacK_2',
 'td_points_2',
 'compilation_2',
 'compilation_3',
 'avg_starts',
 'dpis',
 'dpi_yards',
 'year_1_growth',
 'year_2_growth',
 'categories']

# PCA

In [49]:
## I now want to perform PCA on all the year 1 and 2 stats columns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# I am creating a dataframe of the scaled values of the year 1 and 2 columns
pca_df = scale(df[year_12])

In [50]:
## creating the covariance matrix - this explains the variance between the different
## features within our dataframe

## for example, the value in the i,j position within the matrix explains the variance
## between the ith and the jth elements of a random vector, or between our features
cov_mat = np.cov(pca_df.T)

## creating my eigenvalues and corresponding eigenvectors
eigenValues, eigenVectors = np.linalg.eig(cov_mat)

## creating the eigenpairs - just pairing the eigenvalue with its eigenvector
eigenPairs = [(np.abs(eigenValues[i]), eigenVectors[:,i]) for i in range(len(eigenValues))]

In [51]:
## loop through the eigenpairs and printing out the first row (eigenvalue)
## this is also seen in the code block above but just wanted to loop through again
## as it is a bit more clear like this
## I am also creating a list of the eigenvalues in ascending order to be able to reference it
sort_values = []
for i in eigenPairs:
    print i[0]
    sort_values.append(i[0])

31.1004172447
8.37699654955
5.68144617858
4.66851300718
4.30875326557
3.54751063564
2.85102279075
2.77468351685
2.4530634271
1.98353980126
1.8873361868
1.81354476181
1.46265719107
1.38724426312
1.22216310882
1.14299603062
1.08148128461
1.01553250699
1.00863793646
0.995798092837
0.912404972757
0.935055853353
0.852467904883
0.82253148254
0.778186873741
0.700528141975
0.671373787536
0.641147666505
0.610736725571
0.561725403211
0.538202333045
0.493854700223
0.455623298126
0.429498771074
0.404955434107
0.391863073062
0.364838926175
0.350647228352
0.336641834748
0.325059011605
0.291350446114
0.265907321403
0.259968155731
0.236963353397
0.233805156533
0.196417720057
0.207677376759
0.212154622742
0.167335408622
0.152283964168
0.170452783412
0.141191684846
0.121390989116
0.116682582569
0.107937607097
0.10285217139
0.0744391773946
0.0725376329156
0.0670437039671
0.0570709739462
0.0534423623818
0.0507257642628
0.0468117789775
0.0438166348696
0.0406018410038
0.0260903560814
0.0323898243667
0.03329

In [52]:
## we have the eigenvalues above showing us feature correlation explanation, but it helps
## to see the cumulative variance explained as well, which i can show below

## need to sum the eigen values to get percentages
sumEigenvalues = sum(eigenValues)

## this is a percentage explanation
variance_explained = [(i/sumEigenvalues)*100 for i in sort_values]
variance_explained

[(32.359116279836726+0j),
 (8.7160311480493959+0j),
 (5.9113862069273804+0j),
 (4.8574575081825442+0j),
 (4.4831375362072121+0j),
 (3.6910858223937617+0j),
 (2.9664096554201969+0j),
 (2.8869807711852871+0j),
 (2.5523440426776745+0j),
 (2.0638178121340998+0j),
 (1.9637206358769428+0j),
 (1.8869427173385842+0j),
 (1.5218539915695846+0j),
 (1.4433889444515642+0j),
 (1.2716266100203817+0j),
 (1.18925547432853+0j),
 (1.1252510976830974+0j),
 (1.0566332348866609+0j),
 (1.0494596266412299+0j),
 (1.0361001276527073+0j),
 (0.94933191331187661+0j),
 (0.97289952249415923+0j),
 (0.88696906674481468+0j),
 (0.85582105467789416+0j),
 (0.80968172666764993+0j),
 (0.72887998334834792+0j),
 (0.69854569111239528+0j),
 (0.66709625564496899+0j),
 (0.63545452022681326+0j),
 (0.58445960698173749+0j),
 (0.55998450889069151+0j),
 (0.51384203446880494+0j),
 (0.47406332744135093+0j),
 (0.4468814860535546+0j),
 (0.42134482882640151+0j),
 (0.40772259250433651+0j),
 (0.379604721782626+0j),
 (0.36483865621975553+0j),

In [53]:
### based on the above results, it seems that sticking to 25 features would be a decent
## cutoff point since the variance explained altogether reaches 85%

## this can very easily be manipulated by changing n_components adn then adding/subtracting
## columns to the dataframe in the code block below

## instantiate
pca = PCA(n_components = 25)

## fit and transform the standardized data
pca_model = pca.fit(pca_df)
pca_cols = pca_model.transform(pca_df)

In [54]:
## Here I am simply creating the column headers for the pca features
pca_col_list = []

for i in range(1, 26):
    pca_col_list.append('pca_'+str(i))
    
## going to organize the columns into dataframe for organization
## I am going to set the index of our pca dataframe to the names 
## of the related player
pca_df = pd.DataFrame(pca_cols, index = df.index, columns = pca_col_list)

##previewing dataframe
pca_df.head()

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,...,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,pca_24,pca_25
name,,,,,,,,,,,,,,,,,,,,,
A.J. Green,17.990280,-0.845340,-0.789193,3.363440,4.203103,7.596306,-2.113274,1.899177,5.757547,-0.971711,...,-1.653116,1.170677,-1.117313,-0.451280,0.171508,-1.112356,0.614894,0.165858,-0.974231,-1.352037
A.J. Jenkins,-2.547235,-4.343973,-0.463037,0.386318,-0.153149,2.087280,-0.849217,0.388721,-2.038529,0.955115,...,0.473443,-0.539717,0.133526,-0.226938,-0.082666,-0.080114,0.867716,-0.001003,-0.301392,-0.932536
Aaron Brown,-2.913973,1.173435,-2.452126,0.998614,-1.063707,0.273990,-1.434221,0.739649,0.676427,0.612355,...,-0.153709,-0.274705,-0.150782,0.169129,-0.032974,0.195357,0.228188,0.356468,-0.207825,-0.808390
Aaron Halterman,-4.521102,1.439792,0.563905,2.863056,2.544238,-0.730311,0.568053,0.542730,0.148970,0.423199,...,0.249463,-0.147875,-0.187808,0.034841,0.049696,-0.397459,-0.047224,0.068115,0.074309,-0.125678
Aaron Moorehead,-2.339588,-0.476687,0.514628,0.122931,-1.441598,-1.595380,0.500668,-0.319927,2.211036,0.942728,...,-0.226417,-0.582244,0.154353,0.254678,0.498723,0.644983,0.134924,0.031507,0.010685,-0.193576


In [56]:
## I now want to combine my PCA feature columns with my year 3 data to model
data = pca_df.join(df[year_3])
data.head()

,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,...,yacK_2,td_points_2,compilation_2,compilation_3,avg_starts,dpis,dpi_yards,year_1_growth,year_2_growth,categories
name,,,,,,,,,,,,,,,,,,,,,
A.J. Green,17.990280,-0.845340,-0.789193,3.363440,4.203103,7.596306,-2.113274,1.899177,5.757547,-0.971711,...,123.114306,1.562539,82.935491,59.741763,1.000000,0.0,0.0,9.264454,-13.711468,quality starter
A.J. Jenkins,-2.547235,-4.343973,-0.463037,0.386318,-0.153149,2.087280,-0.849217,0.388721,-2.038529,0.955115,...,0.000000,0.000000,0.000000,0.000000,0.020833,0.0,0.0,0.121706,-0.527123,below average
Aaron Brown,-2.913973,1.173435,-2.452126,0.998614,-1.063707,0.273990,-1.434221,0.739649,0.676427,0.612355,...,0.000000,0.000000,0.000000,0.000000,0.044444,0.0,0.0,-0.205240,0.000000,below average
Aaron Halterman,-4.521102,1.439792,0.563905,2.863056,2.544238,-0.730311,0.568053,0.542730,0.148970,0.423199,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.006153,0.000000,below average
Aaron Moorehead,-2.339588,-0.476687,0.514628,0.122931,-1.441598,-1.595380,0.500668,-0.319927,2.211036,0.942728,...,4.320000,0.000000,0.222518,0.274562,0.000000,0.0,0.0,-0.043790,0.173544,below average


# Model Construction

In [64]:
## I want to build an SVC model to grid search through based on my data df
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

## Defining my features and my target
X = data.drop(['categories', 'compilation_3'], axis = 1)
y = data['categories']

svc = SVC()

svc_params = dict(C=[.5, .7, .85, 1, 1.2],
                 kernel=['rbf', 'linear', 'poly'],
                 degree=[2, 3, 4],
                 gamma=[0,.5,1.0,1.5,2.0])

grid_svc = GridSearchCV(svc, param_grid = svc_params, scoring = 'recall_macro',
                       n_jobs = -1, cv = 3, verbose = 1)

fit_grid = grid_svc.fit(X, y)
print fit_grid.best_score_
print
print fit_grid.best_estimator_

Fitting 3 folds for each of 225 candidates, totalling 675 fits


[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 199 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 449 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.3min remaining:   11.4s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.3min remaining:   11.5s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.3min remaining:   11.5s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.3min remaining:   11.5s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.3min remaining:   11.5s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.4min remaining:   11.5s
[Parallel(n_jobs=-1)]: Done 669 out of 675 | elapsed: 21.5min remaining:   11.6s
[Parallel(n_jobs=-1)]: Done 675 out of 675 | elapsed: 21.5min finished


0.412282338044

SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [66]:
fit_grid.best_score_

0.41228233804378328

In [67]:
from sklearn.metrics import classification_report

ideal_svc = fit_grid.best_estimator_

fit_ideal_svc = ideal_svc.fit(X, y)
preds = fit_ideal_svc.predict(X)
print classification_report(y, preds)


             precision    recall  f1-score   support

    all_pro       0.87      1.00      0.93        27
below average       0.98      0.98      0.98       741
league_average       0.80      0.75      0.77        57
quality starter       0.94      0.96      0.95        47

avg / total       0.96      0.96      0.96       872



In [71]:
from sklearn.cross_validation import cross_val_predict

cvp = cross_val_predict(ideal_svc, X, y, cv = 5, n_jobs = -1, verbose = 1)

print classification_report(y, cvp)

/Users/TerryONeill/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:85: DeprecationWarning: gamma=0.0 has been deprecated in favor of gamma='auto' as of 0.17. Backward compatibility for gamma=0.0 will be removed in 0.18
  DeprecationWarning)
/Users/TerryONeill/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:85: DeprecationWarning: gamma=0.0 has been deprecated in favor of gamma='auto' as of 0.17. Backward compatibility for gamma=0.0 will be removed in 0.18
  DeprecationWarning)
/Users/TerryONeill/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:85: DeprecationWarning: gamma=0.0 has been deprecated in favor of gamma='auto' as of 0.17. Backward compatibility for gamma=0.0 will be removed in 0.18
  DeprecationWarning)
/Users/TerryONeill/anaconda/lib/python2.7/site-packages/sklearn/svm/base.py:85: DeprecationWarning: gamma=0.0 has been deprecated in favor of gamma='auto' as of 0.17. Backward compatibility for gamma=0.0 will be removed in 0.18
  DeprecationWarning)


             precision    recall  f1-score   support

    all_pro       0.26      0.33      0.30        27
below average       0.91      0.91      0.91       741
league_average       0.19      0.18      0.18        57
quality starter       0.14      0.15      0.14        47

avg / total       0.81      0.80      0.80       872



[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.7s finished


In [ ]:
cat_weights = {'all_pro':2, 'below average' : .9, 'league_average' : 5,
                  'quality starter' : 5}

new_svc = SVC(C = .8, class_weight = cat_weights, kernel = 'linear', degree = 2)

cvp = cross_val_predict(new_svc, X, y, cv = 3, n_jobs = -1, verbose = 1)

print classification_report(y, cvp)